# RAGAS 이용한 RAG Pipeline 성능 검증

In [ ]:
!pip install openai llama_index datasets ragas

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

In [ ]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Generator_llm. : Golden Dataset 생성 시 활용하는 모델
generator_llm =
# Critic_llm : 각종 지표에 대한 계산 모델
critic_llm =
embeddings = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 생성할 테스트 쿼리 유형별 분포 정의
distributions = {
    simple:
    reasoning:
    multi_context:
    conditional:
}
generator_naive =


# 제너레이터 이용하여 독스로부터 샘플 쿼리 5개 생성
testset_naive = generator_naive.
testset_naive.to_pandas()

In [ ]:
generator =

language =

#추후 테스트셋 생성에 사용할 제너레이터 오브젝트 한국어에 adaptation 진행
generator.adapt(language, evolutions=[simple, reasoning, conditional, multi_context])
generator.save(evolutions=[simple, reasoning, multi_context, conditional])

In [ ]:
# 생성할 테스트 쿼리 유형별 분포 정의
distributions = {
    simple:0.4,
    reasoning:0.2,
    multi_context:0.2,
    conditional:0.2
}

# 한국어로 Adapt시킨 제너레이터 이용하여 독스로부터 샘플 쿼리 15개 생성
testset = generator.generate_with_llamaindex_docs()

In [ ]:
testset

In [ ]:
test_df = testset.to_pandas()

In [ ]:
# 최종 테스트셋 확인
test_df

# Evaluation

In [ ]:
#평가에 태울 RAG 엔진 생성
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.settings import Settings
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

#벡터 인덱스 설정
vector_index =

#Naive-RAG 엔진 정의
query_engine =


In [ ]:
#평가할 항목 import
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    answer_similarity,
    context_recall,
)

metrics = [
    context_precision,
    context_recall,
    faithfulness,
    answer_similarity,
    answer_relevancy
]


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

#평가용 모델
evaluator_llm = OpenAI()

In [ ]:
# 허깅페이스 데이터셋으로 변환
ds = testset.to_dataset()

In [ ]:
testset.to_pandas()

In [ ]:
# 답변이 안되는 QA페어는 제외
exclude_idx = []

ds_ex = ds.select(
    (
        i for i in range(len(ds))
        if i not in set(exclude_idx)
    )
)

ds_dict = ds_ex.to_dict()

In [ ]:
ds_ex.to_pandas()

In [ ]:
from ragas.integrations.llama_index import evaluate

# Naive-RAG Pipeline으로 테스트셋 질문에 대한 답변 생성 및 사전 import한 평가지표 산출
result = evaluate(
    query_engine=query_engine, # 테스트 대상 RAG
    metrics=metrics, # 평가항목
    dataset=ds_dict, # 테스트셋
    llm=evaluator_llm, # 평가용 모델
    embeddings=embeddings, # 임베딩 모델
    raise_exceptions=False
)


- Faithfulness: Retrieved Context 를 얼마나 잘 참조한 답변인지
- answer_relevancy: Q&A 연관성. 그냥 얼마나 잘 답변했는지
- context_precision: GT의 내용과 연관된 Context간의 순서가 Relevancy 순으로 정렬되어있는지
- context_recall: GT의 내용과 얼마나 context가 relevant 한지
- Answer semantic similarity: Answer vs GT


In [ ]:
# 종합 평가점수 확인
result

In [ ]:
# 세부항목 확인
result.to_pandas()

In [ ]:
print(result.to_pandas().iloc[5]['contexts'])

In [ ]:
print(result.to_pandas().iloc[5]['answer'])

# Rerank 추가한 RAG 비교

In [ ]:
!pip install llama-index-postprocessor-cohere-rerank

In [ ]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

#코히어 리랭크 api 설정
api_key = ''
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [ ]:
# 리랭크가 반영된 Advanced Pipeline 정의
query_engine_cohere =


In [ ]:
result_rerank = evaluate(
    query_engine=query_engine_cohere, # 테스트 대상 RAG
    metrics=metrics, # 평가항목
    dataset=ds_dict, # 테스트셋
    llm=evaluator_llm, # 평가용 모델
    embeddings=embeddings, # 임베딩 모델
    raise_exceptions=False
)


In [ ]:
#naive RAG 전체결과
result

In [ ]:
#advanced RAG 전체결과
result_rerank

In [ ]:
#advanced RAG 세부항목별 결과
result_rerank.to_pandas()

In [ ]:
#naive RAG 세부항목별 결과
result.to_pandas()

In [ ]:
print(result_rerank.to_pandas().iloc[0]['question'])

In [ ]:
print(result_rerank.to_pandas().iloc[0]['answer'])

In [ ]:
print(result.to_pandas().iloc[0]['answer'])

# 오늘 배운 기술 중 택1 해서 평가 해보세요!
- Query Routing
- Query Decomposition
- Query Transformation(HyDE)